## Xarray engine: mono variable with remapping

This notebook demonstrates how to generate an Xarray with a single dataarray containing all the parameters from a GRIB fieldlist. This data structure is often needed for machine learning.

First, we get GRIB data containing multiple forecasts on the surface and pressure levels. We select a single forecast out of it.

In [1]:
import earthkit.data as ekd
ds_fl = ekd.from_source("sample", "mixed_pl_sfc.grib").sel(date=20240603, time=0)

mixed_pl_sfc.grib:   0%|          | 0.00/390k [00:00<?, ?B/s]

In [2]:
ds = ds_fl.to_xarray(fixed_dims=["valid_time", "param", "number"],
                     mono_variable=True,
                     chunks={"valid_time": 1},                    
                     flatten_values=True,                   
                     add_earthkit_attrs=False, 
                     remapping={"param": "{param}_{level}"}
                    )
ds

<xarray.Dataset> Size: 362kB
Dimensions:     (valid_time: 2, param: 32, number: 1, values: 684)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 16B 2024-06-03 2024-06-03T06:00:00
  * param       (param) <U6 768B '2t_0' 'msl_0' 'r_1000' ... 'z_700' 'z_850'
  * number      (number) int64 8B 0
    latitude    (values) float64 5kB dask.array<chunksize=(684,), meta=np.ndarray>
    longitude   (values) float64 5kB dask.array<chunksize=(684,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    data        (valid_time, param, number, values) float64 350kB dask.array<chunksize=(1, 32, 1, 684), meta=np.ndarray>
Attributes:
    paramId:      167
    class:        od
    stream:       oper
    levtype:      sfc
    type:         fc
    expver:       0001
    date:         20240603
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

When generating the Xarray we flattened the field values and chose the chunking so that one chunk would contain all the data belonging to a given valid time.

In [3]:
ds["data"]

<xarray.DataArray 'data' (valid_time: 2, param: 32, number: 1, values: 684)> Size: 350kB
dask.array<open_dataset-data, shape=(2, 32, 1, 684), dtype=float64, chunksize=(1, 32, 1, 684), chunktype=numpy.ndarray>
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 16B 2024-06-03 2024-06-03T06:00:00
  * param       (param) <U6 768B '2t_0' 'msl_0' 'r_1000' ... 'z_700' 'z_850'
  * number      (number) int64 8B 0
    latitude    (values) float64 5kB dask.array<chunksize=(684,), meta=np.ndarray>
    longitude   (values) float64 5kB dask.array<chunksize=(684,), meta=np.ndarray>
Dimensions without coordinates: values
Attributes:
    standard_name:  unknown
    long_name:      2 metre temperature
    units:          K